# Regularized Benders Decomposition - Single Cut
[Murwan Siddig](mailto:msiddig@clemson.edu)

-------------------------------------------------------------------------------------------------------------

### Two-stage stochastic LP (with fixed recourse):

\begin{equation}
\begin{array}{llll}
\displaystyle \min_{x\in\mathbb{R}^{n_1}_+, y\in\mathbb{R}^{n_2}_+} & c^\intercal x+ \mathbb{E}_{\xi}[q_\xi^\intercal y_\xi] \\
\mbox{s.t.} & Ax &\geq b \\
            & T_\xi x +W y_\xi &\geq h_\xi \\
\end{array}
\end{equation}

where
* $A \in \mathbb{R}^{m_1\times n_1}$
* $T_\xi \in \mathbb{R}^{m_2\times n_1}$
* $W \in \mathbb{R}^{m_2\times n_2}$
* $c \in \mathbb{R}^{n_1}$
* $q_\xi \in \mathbb{R}^{n_2}$
* $b \in \mathbb{R}^{m_1}$
* $h_\xi \in \mathbb{R}^{m_2}$

### Feasiblity subproblem:
\begin{equation}
F(\hat x,\xi^s):= \left\{
\begin{array}{llll}
\displaystyle \min_{y_\xi, v^+_\xi,v^-_\xi\in\mathbb{R}^{n_2}_+} & {\bf 1}^\intercal (v^+_\xi+v^-_\xi) \\
\mbox{s.t.} & W y_\xi + v^+_\xi-v^-_\xi\geq h_\xi-T_\xi \hat x \\
\end{array}
\right.
\equiv
\left\{
\begin{array}{llll}
\displaystyle \max_{\lambda_\xi\in\mathbb{R}^{m_2}_+} & (h_\xi-T_\xi \hat x)^\intercal\lambda_\xi \\
\mbox{s.t.} & W^\intercal \lambda_\xi &\leq 0\\
            & +\lambda_\xi &\leq {\bf 1}\\
            & -\lambda_\xi &\leq {\bf 1}\\
\end{array}
\right. 
\end{equation}

### Optimality subproblem:
\begin{equation}
Q(\hat x,\xi^s):= \left\{
\begin{array}{llll}
\displaystyle \min_{y_\xi\in\mathbb{R}^{n_2}_+} & q_\xi^\intercal y_\xi \\
\mbox{s.t.} & W y_\xi \geq h_\xi-T_\xi \hat x \\
\end{array}
\right.
\equiv
\left\{
\begin{array}{llll}
\displaystyle \max_{\pi_\xi\in\mathbb{R}^{m_2}_+} &(h_\xi-T_\xi \hat x)^\intercal\pi_\xi \\
\mbox{s.t.} & W^\intercal \pi_\xi \leq q_\xi\\
\end{array}
\right.
\end{equation}

### Standard Master problem (single-cut):
\begin{equation}
\begin{array}{llll}
\displaystyle \min_{x\in\mathbb{R}^{n_1}_+,\, \theta \in \mathbb{R}} & c^\intercal x+ \theta \\
\mbox{s.t.} & Ax \geq b \\
            & 0\geq \displaystyle \sum_{\forall \xi \in \Xi} (h_\xi-T_\xi \hat x)^\intercal\lambda^j_\xi\times p_\xi & \forall \; j=1, \dots, J_f\\
            & \theta \geq \displaystyle \sum_{\forall \xi \in \Xi} (h_\xi-T_\xi x)^\intercal\pi^j_\xi\times p_\xi & \forall \; j=1, \dots, J_o\\
\end{array}
\end{equation}

where 
* $J_f$ is the number of feasiblity cuts added.
* $J_o$ is the number of optimality cuts added.

### Regularized Master Problem (single-cut):

\begin{equation}
\begin{array}{llll}
\displaystyle \min_{x\in\mathbb{R}^{n_1}_+, \theta\in\mathbb{R}} & c^\intercal x+ \theta + \frac{1}{2}||x - \check x||^2 \\
\mbox{s.t.} & c^\intercal x+ \theta \leq f_{\text{lev}}\\
            & Ax \geq b \\
            & 0\geq \displaystyle (h_\xi-T_\xi \hat x)^\intercal\lambda^j_\xi & \forall \; j=1, \dots, J_f, \forall \xi \in \Xi\\
            & \theta \geq \displaystyle \sum_{\forall \xi \in \Xi} (h_\xi-T_\xi x)^\intercal\pi^j_\xi\times p_\xi & \forall \; j=1, \dots, J_o\\
\end{array}
\end{equation}

where 
* $\check x$ is the incumbent solution. 
* $f_{\text{lev}} := f_{\text{low}} + \gamma(f_{\text{up}}-f_{\text{low}})$ is the level set parameter and $\gamma \in (0,1)$.


In [1]:
import numpy as np
import pandas as pd
from operator import add
import gurobipy as gp;
from gurobipy import GRB;
import numpy as np;

In [2]:
A = [[1, 1, 0, 0]];
T = [[1.0, 1.0, 0.0, 0.0],
     [1.0, 1.0, 0.0, 0.0],
     [0.5, 0.0, 1.0, 0.0],
     [0.5, 0.0, 0.0, 1.0]];
W = [[1, 0, 0, 0],
     [0, 1, 0, 0],
     [0, 0, 1, 0],
     [0, 0, 0, 1]];
c = [[1], [2], [2], [1]];
q = [[1], [2], [1], [2]];
b = [[4]];
h = [[7], [4], [5], [8]];

In [3]:
n1 = len(c); # number of 1st-stage variables
n2 = len(q); # number of 2nd-stage variables
m1 = len(b); # number of 1st-stage constraints
m2 = len(h); # number of 2nd-stage constraints

In [4]:
Ξ = [[1,1/3],
     [4,1],
     [5/2,2/3],
     [1,1]];
N = len(Ξ);

P = [1/N for k in range(N)];

ϵ = 1e-5; #tolarence parameter 

In [5]:
#define functions to modify T according to scenario k
def Tₓ(T,Ξ,n1,m2,k):
    ξ = Ξ[k];
    temp = np.ones((m2,n1));
    temp[0][0] = ξ[0];
    temp[1][0] = ξ[1];
    temp[2][0] = ξ[0];
    temp[3][0] = ξ[1];      
    temp[3][3] = ξ[0];
    return T*temp

#define functions to modify q according to scenario k
def qₓ(q,Ξ,n2,k):
    return q

#define functions to modify h according to scenario k
def hₓ(h,Ξ,m2,k):
    return h

# Solve a mean value problem (MVP) to find an initial lower bound for θ:

In [6]:
ξ̄ = [sum(Ξ[k][n]*P[k] for k in range(N)) for n in range(len(Ξ[0]))];
q̄ = q;
h̄ = h;
T̄ = Tₓ(T,[ξ̄],n1,m2,0)

MVP = gp.Model("MVP");
MVP.modelSense = GRB.MINIMIZE;

x̄ = {};
for j in range(n1):
   x̄[j] = MVP.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=c[j][0]);

ȳ = {};
for j in range(n2):
    ȳ[j] = MVP.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=q̄[j][0]);
    
for i in range(m1):
    MVP.addConstr(sum(A[i][j]*x̄[j] for j in range(n1))>= b[i][0]);
    
for i in range(m2):
    MVP.addConstr(sum(T̄[i][j]*x̄[j] for j in range(n1))+sum(W[i][j]*ȳ[j] for j in range(n2))>= h[i][0]);
    
MVP.update();
MVP.setParam("OutputFlag", 0);
print("===============================================================");
MVP.optimize();
if MVP.status == GRB.OPTIMAL:
    Z̲ = MVP.objVal
    print('MVP objective Z̄ = %g' % Z̲);
    
    x̄val = {};
    for j in range(n1):
        x̄val[j]=x̄[j].x
    
    ȳval = {};
    for j in range(n2):
        ȳval[j]=ȳ[j].x;
else:
    print('No solution');
    print('status = ', MVP.status);

Using license file /Users/murwansiddig/gurobi.lic
Academic license - for non-commercial use only
MVP objective Z̄ = 8.15686


In [7]:
#=================================================================================================================
# Optimality subproblem 
sub_opt = gp.Model("subprob_optimality");
sub_opt.modelSense = GRB.MINIMIZE; 

yᵒ = {};
for j in range(n2):
    yᵒ[j] = sub_opt.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=q[j][0]);

Π = {};
for i in range(m2):
    Π[i] = sub_opt.addConstr(sum(W[i][j]*yᵒ[j] for j in range(n2))>= 0);
    
sub_opt.setParam("OutputFlag", 0);
#=================================================================================================================

#=================================================================================================================
# Feasibility subproblem 
sub_feas = gp.Model("subprob_feasibility");
sub_feas.modelSense = GRB.MINIMIZE; 

yᶠ = {};
v̅ = {};
v̲ = {};

for j in range(n2):
    yᶠ[j] = sub_feas.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=0);
    
for i in range(m2):
    v̅[i] = sub_feas.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=1);
    v̲[i] = sub_feas.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY, obj=1);
    
Λ = {};
for i in range(m2):
    Λ[i] = sub_feas.addConstr(sum(W[i][j]*yᶠ[j] for j in range(n2))+v̅[i]-v̲[i] >= 0);
sub_feas.setParam("OutputFlag", 0);
#=================================================================================================================

In [10]:
#define the level parameter
def f_level(f̲,f̅,γ):
    return f̲+γ*(f̅-f̲)

In [11]:
for x̂ᵏ in [x̄val,np.zeros(n1),np.ones(n1)]:
    #=================================================================================================================
    # Master problem  
    Master_level = gp.Model("Master_single");
    Master_level.modelSense = GRB.MINIMIZE;
    xℓ = {};
    for j in range(n1):
       xℓ[j] = Master_level.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);

    θℓ = Master_level.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY);

    Master_level.addConstr(sum(c[j][0]*xℓ[j] for j in range(n1))+θℓ>= Z̲); #Lower bound for θ
    cons_l = Master_level.addConstr(sum(c[j][0]*xℓ[j] for j in range(n1))+θℓ<= 0); #Level set constraint

    x̂ᵏ = np.zeros(n1); #initialize the new iterate
    x̂ℓ = np.zeros(n1); #initialize the incumbent

    θ̂ℓ = -1e10; #initialize the second stage expected cost;
    Master_level.setParam("OutputFlag", 0);

    print("intitial solution = ", x̂ᵏ)
    #initialize
    LBℓ = 0;
    UBℓ = 1e10;
    γ = 0.2929;
    f̲ = LBℓ;
    f̅ = UBℓ;
    θ̂ℓ = 0;
    Ocuts = 0;
    Fcuts = 0;
    iter = 0;


    while (UBℓ-LBℓ)*1.0/UBℓ > ϵ:
        #set the objective using the incumbent
        Master_level.setObjective(1/2*sum((xℓ[j]-x̂ᵏ[j])*(xℓ[j]-x̂ᵏ[j]) for j in range(n1))+
                                  sum(c[j][0]*xℓ[j] for j in range(n1))+θℓ,GRB.MINIMIZE)

        #Define the level set paramter and update the constraint
        fℓ = f_level(f̲,f̅,γ);
        cons_l.setAttr(GRB.Attr.RHS, fℓ);

        # Solve the master problem
        Master_level.optimize();

        #check
        if Master_level.status != GRB.OPTIMAL:
            #update the level set and break
            print("The level set is empty ==> update f_low")
            f̲ = fℓ;
        elif Master_level.status == GRB.OPTIMAL:
            iter+=1;
            print("new iterate is found")
            for j in range(n1):
                x̂ℓ[j]=xℓ[j].x #value of x
            θ̂ℓ = θℓ.x; #value of θ    
            LBℓ = sum(c[j][0]*x̂ℓ[j] for j in range(n1))+θ̂ℓ; #update the lower bound.

            π = np.zeros((N,m2)); #initialize the optimality dual multipliers (assuming x̂ is feasible ∀ ξ)
            λ = np.zeros((N,m2)); #initialize the feasibility dual multipliers (in case needed);
            Q = np.zeros(N); #initialize a vector for the optimality subproblem optimal objective value ∀ ξ,
            w = np.zeros(N); #initialize a vector for the feasibility subproblem optimal objective value ∀ ξ,
            FLAG = 0; #(0 if x̂ is feasible ∀ ξ, and 1 otherwise)
            for k in range(N):
                Tᵏ = Tₓ(T,Ξ,n1,m2,k);
                qᵏ = qₓ(q,Ξ,n2,k);
                hᵏ = hₓ(h,Ξ,m2,k)
                for i in range(m2):
                    Π[i].setAttr(GRB.Attr.RHS, hᵏ[i][0]-sum(Tᵏ[i][j]*x̂ℓ[j] for j in range(n1)));

                sub_opt.optimize();
                if sub_opt.status == GRB.OPTIMAL:
                    Q[k] = sub_opt.objVal;
                    for i in range(m2):
                        π[k][i] = Π[i].pi;
                elif sub_opt.status == GRB.INFEASIBLE or sub_opt.status == GRB.INF_OR_UNBD:
                    FLAG = 1;
                    break;
                else:
                    print('One of the scenario subproblems is neither optimal or infeasible');
                    print('status = ', sub_opt.status);
                    break
                    exit(0)


            if FLAG == 1:
                for k in range(N):
                    Tᵏ = Tₓ(T,Ξ,n1,m2,k);
                    qᵏ = qₓ(q,Ξ,n2,k);
                    hᵏ = hₓ(h,Ξ,m2,k)
                    for i in range(m2):
                        Λ[i].setAttr(GRB.Attr.RHS, hᵏ[i][0]-sum(Tᵏ[i][j]*x̂ℓ[j] for j in range(n1)));

                    sub_feas.optimize(); 
                    w[k] = sub_feas.objVal; #should always be optimal --  no need to check the status
                    for i in range(m2):
                        λ[k][i] = Λ[i].pi;
                    if w[k] > ϵ:
                        #Add the feasibility cut  
                        Master_level.addConstr(0>= sum(λ[k][i]*(hᵏ[i][0]-sum(Tᵏ[i][j]*xℓ[j] for j in range(n1))) for i in range(m2)));
                        Fcuts +=1;
                        break

            else:
                θ̄ℓ = sum(Q[k]*P[k] for k in range(N));
                if sum(c[j][0]*x̂ℓ[j] for j in range(n1))+θ̄ℓ < UBℓ:
                    UBℓ = sum(c[j][0]*x̂ℓ[j] for j in range(n1))+θ̄ℓ;
                if (UBℓ-LBℓ)/max(1e-10,abs(LBℓ)) < ϵ:
                    break;
                else:
                    Tᵏ = [Tₓ(T,Ξ,n1,m2,k) for k in range(N)];
                    qᵏ = [qₓ(q,Ξ,n2,k) for k in range(N)]
                    hᵏ = [hₓ(h,Ξ,m2,k) for k in range(N)]
                    #Add the optimality cut cut
                    Master_level.addConstr(θℓ>=
                                            sum(
                                            sum(π[k][i]*(hᵏ[k][i][0]-sum(Tᵏ[k][i][j]*xℓ[j] for j in range(n1))) 
                                                for i in range(m2)
                                               )*P[k] 
                                                for k in range(N)
                                                )
                                           ); 
                    Ocuts +=1;
                #Update the incumbent 
                #update the level set parameters f_low and f_up    
                f̲ = LBℓ;
                f̅ = UBℓ;
                x̂ᵏ = x̂ℓ;


            print("iter = ", iter);
            print("Ocuts = ", Ocuts)
            print("Fcuts = ", Fcuts)
            print("FLAG = ", FLAG)
            print("LB = ", LBℓ);
            print("UB = ", UBℓ);
            print("===============================================================");        


    print("#########################################################################################")
    print("iter = ", iter);
    print("# of feasibility cuts added= ", Fcuts);
    print("# of optimality cuts added = ", Ocuts);
    print("LBℓ = ", LBℓ);
    print("UBℓ = ", UBℓ);
    print("x̂ℓ = ", x̂ℓ);
    print("#########################################################################################")
    print(" ")
    print(" ")

intitial solution =  [0. 0. 0. 0.]
new iterate is found
iter =  1
Ocuts =  1
Fcuts =  0
FLAG =  0
LB =  8.156862745098039
UB =  36.0
new iterate is found
iter =  2
Ocuts =  2
Fcuts =  0
FLAG =  0
LB =  8.156862749160348
UB =  16.153998112843436
new iterate is found
iter =  3
Ocuts =  3
Fcuts =  0
FLAG =  0
LB =  10.499223696958191
UB =  14.942077134361119
The level set is empty ==> update f_low
The level set is empty ==> update f_low
The level set is empty ==> update f_low
The level set is empty ==> update f_low
new iterate is found
iter =  4
Ocuts =  4
Fcuts =  0
FLAG =  0
LB =  14.156721844276312
UB =  14.34327815900496
The level set is empty ==> update f_low
new iterate is found
#########################################################################################
iter =  5
# of feasibility cuts added=  0
# of optimality cuts added =  4
LBℓ =  14.25000179561357
UBℓ =  14.250001796833061
x̂ℓ =  [6.99999281e+00 5.04407914e-12 4.98630879e-12 2.89532436e+00]
#########################